<a href="https://colab.research.google.com/github/Euan-J-Austin/Analytics_and_Intelligence/blob/main/RFM_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Check for faulty data!
1. Prepare basket variable
2. Rename variables
3. Define segments
4. Perepare final overview table, each customer having a segment

In [1]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("/content/customer_data.csv")

In [3]:
data.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days
0,22086,777,5/14/2006,9,232
1,2290,1555,09/08/2006,16,115
2,26377,336,11/19/2006,5,43
3,24650,1189,10/29/2006,12,64
4,12883,1229,12/09/2006,12,23


In [4]:
#NB revenue = sales price * no. units sold

In [5]:
data.describe()

,customer_id,revenue,number_of_orders,recency_days
count,39999.000000,39999.000000,39999.000000,39999.000000
mean,20000.000000,938.458461,9.830821,181.412010
std,11546.861045,347.525973,3.127234,131.100023
min,1.000000,11.000000,1.000000,2.000000
25%,10000.500000,690.000000,8.000000,89.000000
50%,20000.000000,913.000000,10.000000,149.000000
75%,29999.500000,1159.000000,12.000000,233.000000
max,39999.000000,2593.000000,24.000000,1032.000000


In [6]:
data['customer_id'].count()

39999

In [7]:
#There's 39999 unique customers, it looks like each customer_id aggregates RFM information about the customer
#Drop most_recent_vist as we have recency days

In [8]:
data = data.drop(['most_recent_visit'], axis = 1)

In [9]:
data.head(0)

,customer_id,revenue,number_of_orders,recency_days


In [10]:
data['Monetary'] = data.revenue / data.number_of_orders
data.head(0)

,customer_id,revenue,number_of_orders,recency_days,Monetary


In [11]:
data = data.drop(columns = 'revenue')
data.head(0)

,customer_id,number_of_orders,recency_days,Monetary


In [12]:
data = data.rename(columns = {'recency_days': 'Recency',
                              'number_of_orders': 'Frequency'})

data.head(0)

,customer_id,Frequency,Recency,Monetary


In [13]:
#I'm going to move Recency to the first position to match the acronym

In [14]:
df = pd.DataFrame(data)
first_col = df.pop('Recency')
df.insert(1, 'Recency', first_col)

In [16]:
df.head(0)

,customer_id,Recency,Frequency,Monetary


In [ ]:
#Dividing into quartiles

In [17]:
df['R'] = pd.qcut(x = df['Recency'], q = 4, labels = range(4, 0, -1))
df['F'] = pd.qcut(x = df['Frequency'], q = 4, labels = range(1, 5, 1))
df['M'] = pd.qcut(x = df['Monetary'], q = 4, labels = range(1, 5, 1))
df.head()

,customer_id,Recency,Frequency,Monetary,R,F,M
0,22086,232,9,86.333333,2,2,2
1,2290,115,16,97.187500,3,4,3
2,26377,43,5,67.200000,4,1,1
3,24650,64,12,99.083333,4,3,3
4,12883,23,12,102.416667,4,3,3


In [18]:
df['RFM_Score'] = df[['R', 'F', 'M']].sum(axis = 1)
df.head()

,customer_id,Recency,Frequency,Monetary,R,F,M,RFM_Score
0,22086,232,9,86.333333,2,2,2,6
1,2290,115,16,97.187500,3,4,3,10
2,26377,43,5,67.200000,4,1,1,6
3,24650,64,12,99.083333,4,3,3,10
4,12883,23,12,102.416667,4,3,3,10


In [20]:
df.tail()

,customer_id,Recency,Frequency,Monetary,R,F,M,RFM_Score
39994,3249,31,10,99.800000,4,2,3,9
39995,6686,187,8,96.375000,2,1,3,6
39996,16418,154,9,112.888889,2,2,4,8
39997,9117,195,7,96.857143,2,1,3,6
39998,19184,113,13,116.846154,3,4,4,11


In [25]:
def rfm_segment(df):
  if df['RFM_Score'] >= 11:
    return 'Superstar'
  elif ((df['RFM_Score'] >= 8) and ((df['RFM_Score'] < 11))):
    return 'Future Champion'
  elif ((df['RFM_Score'] >= 6) and ((df['RFM_Score'] < 8))):
    return 'High Potential'
  else:
    return 'Low Relevance'

In [26]:
df['RFM_level'] = df.apply(rfm_segment, axis = 1)
df.head()

,customer_id,Recency,Frequency,Monetary,R,F,M,RFM_Score,RFM_level
0,22086,232,9,86.333333,2,2,2,6,High Potential
1,2290,115,16,97.187500,3,4,3,10,Future Champion
2,26377,43,5,67.200000,4,1,1,6,High Potential
3,24650,64,12,99.083333,4,3,3,10,Future Champion
4,12883,23,12,102.416667,4,3,3,10,Future Champion


In [37]:
RFM_level_tbl = df.groupby("RFM_level").agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(2)

RFM_level_tbl

Recency Frequency Monetary       
                   mean      mean     mean  count
RFM_level                                        
Future Champion  115.58     11.37   101.65  15916
High Potential   204.71      8.88    95.34  13200
Low Relevance    301.40      7.29    79.51   8494
Superstar         64.67     13.84   111.72   2389

In [38]:
total_customers = RFM_level_tbl['Monetary']['count'].sum() # can use this to add proportion column

In [39]:
RFM_level_tbl['Proportion'] = round((RFM_level_tbl['Monetary']['count']/total_customers)*100, 2)

In [40]:
RFM_level_tbl

Recency Frequency Monetary        Proportion
                   mean      mean     mean  count           
RFM_level                                                   
Future Champion  115.58     11.37   101.65  15916      39.79
High Potential   204.71      8.88    95.34  13200      33.00
Low Relevance    301.40      7.29    79.51   8494      21.24
Superstar         64.67     13.84   111.72   2389       5.97

In [36]:
data.describe()

,customer_id,Frequency,Recency,Monetary
count,39999.000000,39999.000000,39999.000000,39999.000000
mean,20000.000000,9.830821,181.412010,95.467604
std,11546.861045,3.127234,131.100023,19.387337
min,1.000000,1.000000,2.000000,11.000000
25%,10000.500000,8.000000,89.000000,82.531373
50%,20000.000000,10.000000,149.000000,95.111111
75%,29999.500000,12.000000,233.000000,107.750000
max,39999.000000,24.000000,1032.000000,207.333333
